# Sparse Autoencoder Training Demo

## Setup

In [26]:
# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import os

import torch
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import PreTrainedTokenizerBase

from sparse_autoencoder import SparseAutoencoder, TensorActivationStore, pipeline
from sparse_autoencoder.source_data.pile_uncopyrighted import PileUncopyrightedDataset


os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [28]:
device = get_device()

### Source Model

In [29]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype="float32")
src_d_mlp: int = src_model.cfg.d_mlp  # type: ignore
src_d_mlp

Loaded pretrained model solu-1l into HookedTransformer


2048

### Source Dataset

In [30]:
tokenizer: PreTrainedTokenizerBase = src_model.tokenizer  # type: ignore
source_data = PileUncopyrightedDataset(tokenizer=tokenizer)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

### Activation Store

In [31]:
max_items = 1_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [32]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [33]:
# wbandb.init(project="sparse-autoencoder", dir=".cache/wandb")

In [34]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    source_dataset=source_data,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
    max_activations=100_000_000,
)

Total activations trained on:   0%|          | 0/100000000 [00:00<?, ?it/s, Generate/train iterations=0]

Generate Activations:   0%|          | 0/1000000 [00:00<?, ?it/s]